# oura

> main module with qself functionality for the Oura API

In [1]:
#| default_exp oura

In [2]:
#| export
from datetime import date, datetime, timedelta
import logging
import os

from dotenv import load_dotenv
import requests

from qself.oura_models import OuraGenericResponse, OuraWorkoutResponse

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [4]:
# | export
class OuraAPIClient:

    ENDPOINT_TO_API_VERSION = {
        "activity": "v1",
        "bedtime": "v1",
        "daily_activity": "v2",
        "heartrate": "v2",
        "personal_info": "v2",
        "readiness": "v1",
        "session": "v2",
        "sleep": "v1",
        "tag": "v2",
        "workout": "v2",
    }

    ENDPOINT_TO_RESPONSE_MODEL = {
        "activity": OuraGenericResponse,
        "bedtime": OuraGenericResponse,
        "daily_activity": OuraGenericResponse,
        "heartrate": OuraGenericResponse,
        "personal_info": OuraGenericResponse,
        "readiness": OuraGenericResponse,
        "session": OuraGenericResponse,
        "sleep": OuraGenericResponse,
        "tag": OuraGenericResponse,
        "workout": OuraWorkoutResponse,
    }

    API_VERSION_TO_BASE_URL = {
        "v1": "https://api.ouraring.com/v1",
        "v2": "https://api.ouraring.com/v2/usercollection",
    }

    API_VERSION_TO_DATE_POSTFIX = {"v1": "", "v2": "_date"}

    def __init__(self, personal_token):
        self.personal_token = personal_token

    def __call__(
        self, endpoint: str, start: str = None, end: str = None, *, next_token=None, i=0
    ) -> OuraGenericResponse:
        api_version = self.ENDPOINT_TO_API_VERSION[endpoint]
        if api_version != "v2" and next_token is not None:
            raise ValueError("Only v2 API supports next_token argument.")
        base_url = self.API_VERSION_TO_BASE_URL[api_version]
        start_param = f"start{self.API_VERSION_TO_DATE_POSTFIX[api_version]}"
        end_param = f"end{self.API_VERSION_TO_DATE_POSTFIX[api_version]}"
        url = f"{base_url}/{endpoint}"  # TODO more robust URL joining
        if start is None:
            params = None if end is None else {end_param: end}
        else:
            params = {
                start_param: start,
                end_param: end,
            }  # TODO what if end is None? test this
        if next_token is not None:
            params["next_token"] = next_token
        headers = {"Authorization": f"Bearer {self.personal_token}"}
        response = requests.request("GET", url, headers=headers, params=params)
        j = response.json()
        model = self.ENDPOINT_TO_RESPONSE_MODEL[endpoint].parse_obj(j)
        if model.next_token is not None:
            logger.debug(
                f"Using continuation token {i}, last_date {model.data[-1].day}: {model.next_token}"
            )
            new_start = (model.data[-1].day + timedelta(days=1)).isoformat()
            j_new = self(endpoint, new_start, end, next_token=model.next_token, i=i + 1)
            model.data.extend(j_new.data)
        return model

In [5]:
#| hide
load_dotenv()

True

In [6]:
client = OuraAPIClient(os.environ["OURA_PERSONAL_ACCESS_TOKEN"])

In [7]:
wo = client("workout", "2022-01-02", "2022-02-01")

In [8]:
da = client("daily_activity", "2022-01-01", "2022-01-02")

In [9]:
assert da.data[0].day == date(2022, 1, 1)

In [10]:
for d in da.data:
    print(sorted(d.dict().keys()))

['active_calories', 'average_met_minutes', 'class_5_min', 'contributors', 'day', 'equivalent_walking_distance', 'high_activity_met_minutes', 'high_activity_time', 'inactivity_alerts', 'low_activity_met_minutes', 'low_activity_time', 'medium_activity_met_minutes', 'medium_activity_time', 'met', 'meters_to_target', 'non_wear_time', 'resting_time', 'score', 'sedentary_met_minutes', 'sedentary_time', 'steps', 'target_calories', 'target_meters', 'timestamp', 'total_calories']


Longer time frames are supported, too, using automatically handled continuation tokens:

In [11]:
da = client("daily_activity", "2010-01-01", "2022-06-30")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()